In [95]:
import pandas as pd
from datetime import datetime
from glob import glob

In [250]:
def findJoinColumn(columns):
    # check for various options used for join columns in data sets
    # return whicever is present
    # pray no more than 1 is used in the same dataset
    colOptions = ['GEOID', 'fips', 'countyFIPS', 'stateFIPS']
    for option in colOptions:
        if option in columns:
            return option

In [251]:
def findDateIndex(columns):
    for column in columns:
        if column.count('-') > 1 or column.count('/') > 1:
            return columns.index(column)

In [252]:
def convertToUsaFacts(columns):
    mapping = {}
    
    for column in columns:
        if '-' not in column:
            mapping[column] = column
        else:
            parts = column.split('-')
            year = parts[0]
            month = parts[1]
            day = parts[2]
            mapping[column] = f"{int(month)}/{int(day)}/{year[2:]}"
    
    return mapping

In [262]:
raw = pd.read_csv("https://raw.githubusercontent.com/COVIDExposureIndices/COVIDExposureIndices/master/dex_data/county_dex.csv")

In [265]:
raw = pd.read_csv("https://raw.githubusercontent.com/COVIDExposureIndices/COVIDExposureIndices/master/dex_data/state_dex.csv")
        

In [267]:
for column in raw.columns:
    print(column)

state
date
dex
num_devices
dex_a
num_devices_a
dex_income_1
num_devices_income_1
dex_income_1_a
num_devices_income_1_a
dex_income_2
num_devices_income_2
dex_income_2_a
num_devices_income_2_a
dex_income_3
num_devices_income_3
dex_income_3_a
num_devices_income_3_a
dex_income_4
num_devices_income_4
dex_income_4_a
num_devices_income_4_a
dex_education_1
num_devices_education_1
dex_education_1_a
num_devices_education_1_a
dex_education_2
num_devices_education_2
dex_education_2_a
num_devices_education_2_a
dex_education_3
num_devices_education_3
dex_education_3_a
num_devices_education_3_a
dex_education_4
num_devices_education_4
dex_education_4_a
num_devices_education_4_a
dex_race_asian
num_devices_race_asian
dex_race_asian_a
num_devices_race_asian_a
dex_race_black
num_devices_race_black
dex_race_black_a
num_devices_race_black_a
dex_race_hispanic
num_devices_race_hispanic
dex_race_hispanic_a
num_devices_race_hispanic_a
dex_race_white
num_devices_race_white
dex_race_white_a
num_devices_race_white

In [253]:
def getDexData(geography):
    # 
    if (geography == "county"):
        raw = pd.read_csv("https://raw.githubusercontent.com/COVIDExposureIndices/COVIDExposureIndices/master/dex_data/county_dex.csv")
        joinCol = 'county'
    else:
        raw = pd.read_csv("https://raw.githubusercontent.com/COVIDExposureIndices/COVIDExposureIndices/master/dex_data/state_dex.csv")
        joinCol = 'GEOID'
        geoidTable = pd.read_csv('./statename_geoid.csv')
        raw = raw.merge(geoidTable, left_on="state", right_on="STUSPS", how="left")
        
    raw['dex'] = raw['dex'].round(1)
    raw['dex_a'] = raw['dex_a'].round(1)

    dex = raw[[joinCol,'date','dex']]
    dex_a = raw[[joinCol,'date','dex_a']]

    # thanks to @piRSquared on stackoverflow for this nifty pivot expressions
    # https://stackoverflow.com/questions/54915215/expressing-time-series-data-in-the-columns-rather-than-the-rows-of-a-dataframe
    dex = dex.pivot_table(index=joinCol, columns='date').swaplevel(0, 1, 1).sort_index(1).reset_index()
    dex.columns = [column[0] for column in list(dex.columns)]

    dex_a = dex_a.pivot_table(index=joinCol, columns='date').swaplevel(0, 1, 1).sort_index(1).reset_index()
    dex_a.columns = [column[0] for column in list(dex_a.columns)]

    return { 'dex': dex, 'dex_a': dex_a}

In [256]:
def parseDex(fileName, dataFrame, dexJoinColumn):
    casesDf = pd.read_csv(fileName)

    dateColumnIndex = findDateIndex(list(casesDf.columns))
    nonDateCols = list(casesDf.columns)[:dateColumnIndex]
    dateCols = list(casesDf.columns)[dateColumnIndex:]
    dateCols.sort()

    joinColumn = findJoinColumn(list(casesDf.columns))
    casesDf = casesDf[nonDateCols]
    
    if 'usafacts' in fileName:
        dataFrame = dataFrame.rename(columns=convertToUsaFacts(list(dataFrame.columns)))
        
    mergedDf = dataFrame.merge(casesDf, left_on=dexJoinColumn, right_on=joinColumn)

    for column in dateCols:
        if column not in mergedDf.columns:
            mergedDf[column] = None
    parsedDf = mergedDf[nonDateCols + dateCols]

    fileName = fileName.split('confirmed_')[1].split('.csv')[0]
    
    return {'fileName': fileName, 'df': parsedDf}

In [259]:
dex_county

,county,2020-01-20,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,...,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30
0,1001,217.5,119.0,128.8,135.8,231.6,282.2,161.4,118.9,118.2,...,241.1,269.7,373.6,317.6,19.3,387.3,191.3,168.2,137.1,156.4
1,1003,166.3,134.1,127.7,135.0,193.7,206.7,139.1,127.2,121.6,...,206.8,221.4,258.8,220.7,23.0,289.8,180.4,166.7,141.1,151.4
2,1005,85.2,56.2,64.4,75.4,107.6,118.0,64.3,63.1,56.5,...,103.2,124.5,158.7,123.4,13.4,162.4,83.5,82.7,58.5,72.5
3,1007,159.3,88.8,89.3,100.5,156.1,190.3,111.4,81.4,102.9,...,169.5,212.6,255.0,181.2,24.2,270.5,119.0,111.2,92.9,105.8
4,1009,156.4,110.6,127.3,117.6,196.8,207.7,133.0,117.6,113.3,...,247.7,263.1,331.1,264.7,28.1,294.9,178.4,150.2,131.4,150.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,56029,24.5,52.1,32.1,40.1,43.9,51.9,47.3,23.8,35.7,...,40.6,26.6,39.1,29.4,13.6,57.1,46.8,27.4,33.0,34.0
2014,56033,42.9,30.3,35.1,41.4,49.3,37.6,51.2,25.0,24.4,...,39.9,38.5,51.7,56.2,6.7,85.8,25.9,27.3,25.4,28.0
2015,56037,67.7,63.0,53.6,64.9,78.7,76.3,59.2,46.9,54.6,...,53.5,59.3,76.1,79.6,22.2,81.4,56.8,44.0,33.3,41.4
2016,56039,95.9,231.6,177.5,217.5,150.2,131.9,183.7,169.5,109.0,...,176.0,125.3,266.8,173.0,74.5,401.0,298.4,185.0,126.1,118.2


In [258]:
county = getDexData("county")

dex_county = county['dex']
dex_a_county = county['dex_a']

state = getDexData("state")

dex_state = state['dex']
dex_a_state = state['dex_a']

fileList = glob('../../docs/csv/*confirmed*.csv')

for file in fileList:
    if 'state' in file:
        dexJoinColumn = 'GEOID'
        dataFrame1 = dex_state
        dataFrame2 = dex_a_state
    else:
        dexJoinColumn = 'county'
        dataFrame1 = dex_county
        dataFrame2 = dex_a_county
        
    parsed1 = parseDex(file, dataFrame1, dexJoinColumn)
    parsed2 = parseDex(file, dataFrame2, dexJoinColumn)
    
    parsed1['df'].to_csv(f'../../docs/csv/covid_county_dex_{parsed1["fileName"]}.csv',index=False)
    parsed2['df'].to_csv(f'../../docs/csv/covid_county_dex_a_{parsed2["fileName"]}.csv',index=False)
    parsed1['df'].to_csv(f'./csv/covid_county_dex_{parsed1["fileName"]}.csv',index=False)
    parsed2['df'].to_csv(f'./csv/covid_county_dex_a_{parsed2["fileName"]}.csv',index=False)